<a href="https://colab.research.google.com/github/Maheenms/GoogleCoLab/blob/main/demographics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.2'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Demographics").getOrCreate()

In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.1/22-big-data/day_1/demographics.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("demographics.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+---+-----------------+---+--------+---------+--------+--------------------+---------------+------+--------------------+
| id|             name|age|height_m|weight_kg|children|          occupation|academic_degree|salary|            location|
+---+-----------------+---+--------+---------+--------+--------------------+---------------+------+--------------------+
|  1|    Glad Gavrieli| 38|    1.52|       74|       0|Computer Systems ...|       Bachelor|    78|           Louisiana|
|  2|  Henrieta Fittes| 34|    1.72|       39|       4|             Teacher|         Master|    44|            Illinois|
|  3|   Peyton Dulanty| 24|     1.8|       47|       5|Senior Quality En...|            PhD|    44|      North Carolina|
|  4|     Denna Morgen| 48|    1.81|       71|       5|   Account Executive|         Master|    81|          California|
|  5|    Camella Izaks| 34|    1.65|       60|       1|   Director of Sales|            PhD|    76|                Ohio|
|  6|     Shara Esposi| 49|     

In [4]:
# Print the column names
df.columns

['id',
 'name',
 'age',
 'height_m',
 'weight_kg',
 'children',
 'occupation',
 'academic_degree',
 'salary',
 'location']

In [5]:
# Print out the first 10 rows
df.show(10)

+---+-----------------+---+--------+---------+--------+--------------------+---------------+------+--------------------+
| id|             name|age|height_m|weight_kg|children|          occupation|academic_degree|salary|            location|
+---+-----------------+---+--------+---------+--------+--------------------+---------------+------+--------------------+
|  1|    Glad Gavrieli| 38|    1.52|       74|       0|Computer Systems ...|       Bachelor|    78|           Louisiana|
|  2|  Henrieta Fittes| 34|    1.72|       39|       4|             Teacher|         Master|    44|            Illinois|
|  3|   Peyton Dulanty| 24|     1.8|       47|       5|Senior Quality En...|            PhD|    44|      North Carolina|
|  4|     Denna Morgen| 48|    1.81|       71|       5|   Account Executive|         Master|    81|          California|
|  5|    Camella Izaks| 34|    1.65|       60|       1|   Director of Sales|            PhD|    76|                Ohio|
|  6|     Shara Esposi| 49|     

In [6]:
# Select the age, height_m, and weight_kg columns and use describe to show the summary statistics
df.select("age","height_m","weight_kg").describe().show()

+-------+------------------+-------------------+------------------+
|summary|               age|           height_m|         weight_kg|
+-------+------------------+-------------------+------------------+
|  count|              1000|               1000|              1000|
|   mean|            42.961|  1.750380000000002|            64.308|
| stddev|14.166869067623207|0.14157152997018183|15.840077147667067|
|    min|                18|                1.5|                38|
|    max|                67|                2.0|                90|
+-------+------------------+-------------------+------------------+



In [7]:
# Print the schema to see the types
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- height_m: string (nullable = true)
 |-- weight_kg: string (nullable = true)
 |-- children: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- academic_degree: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- location: string (nullable = true)



In [9]:
# Rename the Salary column to `Salary (1k)` and show only this new column
df.withColumnRenamed('salary','Salary (1k)').select("Salary (1k)").show()

+-----------+
|Salary (1k)|
+-----------+
|         78|
|         44|
|         44|
|         81|
|         76|
|         68|
|         85|
|         84|
|         88|
|        116|
|         51|
|         66|
|         90|
|         40|
|         96|
|        116|
|         74|
|        103|
|         46|
|        114|
+-----------+
only showing top 20 rows



In [10]:
df = df.withColumnRenamed('salary','Salary (1k)')
df.show()

+---+-----------------+---+--------+---------+--------+--------------------+---------------+-----------+--------------------+
| id|             name|age|height_m|weight_kg|children|          occupation|academic_degree|Salary (1k)|            location|
+---+-----------------+---+--------+---------+--------+--------------------+---------------+-----------+--------------------+
|  1|    Glad Gavrieli| 38|    1.52|       74|       0|Computer Systems ...|       Bachelor|         78|           Louisiana|
|  2|  Henrieta Fittes| 34|    1.72|       39|       4|             Teacher|         Master|         44|            Illinois|
|  3|   Peyton Dulanty| 24|     1.8|       47|       5|Senior Quality En...|            PhD|         44|      North Carolina|
|  4|     Denna Morgen| 48|    1.81|       71|       5|   Account Executive|         Master|         81|          California|
|  5|    Camella Izaks| 34|    1.65|       60|       1|   Director of Sales|            PhD|         76|              

In [15]:
# Create a new column called `Salary` where the values are the `Salary (1k)` * 1000
# Show the columns `Salary` and `Salary (1k)`

df = df.withColumn('Salary', df['Salary (1k)'] * 1000)

df.show(5)

+---+---------------+---+--------+---------+--------+--------------------+---------------+-----------+--------------+-------+
| id|           name|age|height_m|weight_kg|children|          occupation|academic_degree|Salary (1k)|      location| Salary|
+---+---------------+---+--------+---------+--------+--------------------+---------------+-----------+--------------+-------+
|  1|  Glad Gavrieli| 38|    1.52|       74|       0|Computer Systems ...|       Bachelor|         78|     Louisiana|78000.0|
|  2|Henrieta Fittes| 34|    1.72|       39|       4|             Teacher|         Master|         44|      Illinois|44000.0|
|  3| Peyton Dulanty| 24|     1.8|       47|       5|Senior Quality En...|            PhD|         44|North Carolina|44000.0|
|  4|   Denna Morgen| 48|    1.81|       71|       5|   Account Executive|         Master|         81|    California|81000.0|
|  5|  Camella Izaks| 34|    1.65|       60|       1|   Director of Sales|            PhD|         76|          Ohio|7

In [16]:
df.select(["Salary", "Salary (1k)"]).show(truncate=False)

+--------+-----------+
|Salary  |Salary (1k)|
+--------+-----------+
|78000.0 |78         |
|44000.0 |44         |
|44000.0 |44         |
|81000.0 |81         |
|76000.0 |76         |
|68000.0 |68         |
|85000.0 |85         |
|84000.0 |84         |
|88000.0 |88         |
|116000.0|116        |
|51000.0 |51         |
|66000.0 |66         |
|90000.0 |90         |
|40000.0 |40         |
|96000.0 |96         |
|116000.0|116        |
|74000.0 |74         |
|103000.0|103        |
|46000.0 |46         |
|114000.0|114        |
+--------+-----------+
only showing top 20 rows

